In [126]:
import pandas as pd
from flask import  Flask, request, jsonify, render_template
from flask_cors import CORS, cross_origin
import json

In [127]:
app = Flask('Nuesta API')
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'

In [128]:
df = pd.read_csv('./Data/PVR_DAM_all.csv')
dam = df[df['punto_tipo'] == 'Intersercción']
pvr = df[df['punto_tipo'] != 'Intersercción']

In [129]:
def find_closest(df, x, y):
    dist = (df['x'] - x).abs() + (df['y'] - y).abs()
    return df.loc[dist.idxmin()]

In [130]:
@app.route('/predict',methods=['POST'])
@cross_origin()
def predict():
    data = request.get_json(force=True)
    predict = find_closest(dam, float(data['lng']), float(data['lat']))
    return predict.to_json()


In [131]:
@app.route('/damero',methods=['GET'])
@cross_origin()
def damero():
    return dam.to_json(orient = 'records')

In [132]:
@app.route('/pvr',methods=['GET'])
@cross_origin()
def pvrendpoint():
    return pvr.to_json(orient = 'records')

In [133]:
app.run(host='0.0.0.0', port=5009)

 * Serving Flask app "Nuesta API" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.0.68:5009/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Sep/2022 00:02:04] "GET /damero HTTP/1.1" 200 -
127.0.0.1 - - [13/Sep/2022 00:02:19] "GET /damero HTTP/1.1" 200 -
127.0.0.1 - - [13/Sep/2022 00:02:21] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [13/Sep/2022 00:02:21] "POST /predict HTTP/1.1" 400 -
127.0.0.1 - - [13/Sep/2022 00:02:22] "POST /predict HTTP/1.1" 400 -
127.0.0.1 - - [13/Sep/2022 00:02:24] "POST /predict HTTP/1.1" 400 -
127.0.0.1 - - [13/Sep/2022 00:02:41] "GET /damero HTTP/1.1" 200 -
127.0.0.1 - - [13/Sep/2022 00:02:51] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [13/Sep/2022 00:02:51] "POST /predict HTTP/1.1" 400 -
127.0.0.1 - - [13/Sep/2022 00:02:52] "POST /predict HTTP/1.1" 400 -
127.0.0.1 - - [13/Sep/2022 00:03:13] "GET /damero HTTP/1.1" 200 -
127.0.0.1 - - [13/Sep/2022 00:03:14] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [13/Sep/2022 00:03:14] "POST /predict HTTP/1.1" 400 -
127.0.0.1 - - [13/Sep/2